# Multiple Instance Learning (MIL) for Slide Classification

End-to-end example using pre-extracted patch features and an attention-based MIL head to predict slide labels.

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
import numpy as np
# Synthetic bag features and labels
bags = [torch.randn(128, 512) for _ in range(64)]
labels = torch.randint(0,2,(64,))
class AttnMIL(nn.Module):
    def __init__(self, d=512, k=128):
        super().__init__()
        self.attn = nn.Sequential(nn.Linear(d,k), nn.Tanh(), nn.Linear(k,1))
        self.clf = nn.Linear(d,2)
    def forward(self, x):
        a = self.attn(x)
        w = torch.softmax(a, dim=0)
        z = (w * x).sum(dim=0)
        return self.clf(z)
model = AttnMIL()
opt = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
for epoch in range(5):
    tot=0
    for x, y in zip(bags, labels):
        opt.zero_grad(); logits = model(x); loss = loss_fn(logits.unsqueeze(0), y.unsqueeze(0)); loss.backward(); opt.step(); tot+=loss.item()
    print('epoch', epoch, 'loss', round(tot/len(bags),4))